In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

**Key Performance Indicators**

H₀: There are no risk differences across provinces
KPIs: Claim Frequency, Claim Severity

H₀: There are no risk differences between zip codes
KPIs: Claim Frequency, Claim Severity

H₀: There are no significant margin (profit) difference between zip codes
KPI: Margin (TotalPremium - TotalClaims)

H₀: There are not significant risk difference between Women and Men
KPIs: Claim Frequency, Claim Severity

In [ ]:
#Loading preprocessed insurance data
Insurance_processed_df = pd.read_csv(r'C:\Users\USER\10 Academy\AlphaCare-Insurance-Risk-Analytics-Predictive-Modeling-End-to-End-\data\processed\cleaned_insurance_data.csv')

C:\Users\USER\AppData\Local\Temp\ipykernel_10876\3693883365.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  Insurance_processed_df = pd.read_csv(r'C:\Users\USER\10 Academy\AlphaCare-Insurance-Risk-Analytics-Predictive-Modeling-End-to-End-\data\processed\cleaned_insurance_data.csv')


Province
gauteng          293756
kwazulu-natal    124692
western cape     123183
north west        96071
mpumalanga        26436
eastern cape      24785
limpopo           16658
northern cape      5551
free state         4756
Name: count, dtype: int64

In [10]:
# Simulate claims: make KWAZULU-NATAL slightly higher risk for demonstration
n_policies = len(Insurance_processed_df)
Insurance_processed_df['HasClaim'] = np.random.binomial(1, 0.15, n_policies) # Baseline claim rate
Insurance_processed_df.loc[Insurance_processed_df['Province'] == 'KWAZULU-NATAL', 'HasClaim'] = np.random.binomial(1, 0.18, Insurance_processed_df[Insurance_processed_df['Province'] == 'KWAZULU-NATAL'].shape[0])

# Simulate TotalClaims amount (only for policies with claims)
Insurance_processed_df['TotalClaims'] = Insurance_processed_df['HasClaim'] * np.random.normal(5000, 1500, n_policies)
Insurance_processed_df.loc[Insurance_processed_df['TotalClaims'] < 0, 'TotalClaims'] = 0 # Ensure no negative claims

# Calculate Margin per policy
Insurance_processed_df['Margin'] = Insurance_processed_df['TotalPremium'] - Insurance_processed_df['TotalClaims']

print("Original DataFrame head:")
print(Insurance_processed_df.head())
print("\nProvince value counts:")
print(Insurance_processed_df['Province'].value_counts())
# Step 2: Create Control (Group A) and Test (Group B) DataFrames
# Now, filter your Insurance_df to create the two groups you want to compare for this specific test.



# Select the two provinces for comparison
province_A = 'GAUTENG'
province_B = 'KWAZULU-NATAL'

# Create Group A (Control)
group_A_df = Insurance_processed_df[Insurance_processed_df['Province'] == province_A].copy()
group_A_df['Group'] = 'A' # Add a group identifier

# Create Group B (Test)
group_B_df = Insurance_processed_df[Insurance_processed_df['Province'] == province_B].copy()
group_B_df['Group'] = 'B' # Add a group identifier

# Combine for easier pre-analysis (optional, but often helpful)
comparison_df = pd.concat([group_A_df, group_B_df])

print(f"\nGroup A ({province_A}) policies: {len(group_A_df)}")
print(f"Group B ({province_B}) policies: {len(group_B_df)}")
print("\nCombined comparison_df head:")
print(comparison_df.head())

Original DataFrame head:
  TransactionMonth  UnderwrittenCoverID  PolicyID  IsVATRegistered  \
0       2015-03-01               145249     12827             True   
1       2015-05-01               145249     12827             True   
2       2015-07-01               145249     12827             True   
3       2015-05-01               145255     12827             True   
4       2015-07-01               145255     12827             True   

  Citizenship          LegalType Title Language                 Bank  \
0         NaN  close corporation    mr  english  first national bank   
1         NaN  close corporation    mr  english  first national bank   
2         NaN  close corporation    mr  english  first national bank   
3         NaN  close corporation    mr  english  first national bank   
4         NaN  close corporation    mr  english  first national bank   

       AccountType  ...              Section                          Product  \
0  current account  ...  motor comprehen

In [14]:
numerical_confounding_vars = ['PolicyholderAge'] # Add others as needed

for var in numerical_confounding_vars:
    print(f"\nChecking balance for: {var}")
    mean_A = group_A_df[var].mean()
    std_A = group_A_df[var].std()
    mean_B = group_B_df[var].mean()
    std_B = group_B_df[var].std()

    print(f"  Group A ({province_A}) - Mean {var}: {mean_A:.2f}, Std Dev: {std_A:.2f}")
    print(f"  Group B ({province_B}) - Mean {var}: {mean_B:.2f}, Std Dev: {std_B:.2f}")

    # Perform Independent Samples t-test
    stat, p_value = stats.ttest_ind(group_A_df[var], group_B_df[var], equal_var=False) # Welch's t-test, generally safer

    print(f"  T-test p-value: {p_value:.4f}")
    if p_value > 0.05:
        print(f"  Result: Groups are statistically similar on {var} (Fail to reject H₀). Good!")
    else:
        print(f"  Result: Groups are statistically different on {var} (Reject H₀). Concern! Requires further investigation or adjustment.")

    sns.histplot(data=comparison_df, x=var, hue='Group', kde=True)
    plt.title(f'Distribution of {var} by Group')
    plt.show()


Checking balance for: PolicyholderAge


KeyError: 'PolicyholderAge'

In [12]:
Insurance_processed_df.columns

Index(['TransactionMonth', 'UnderwrittenCoverID', 'PolicyID',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode',
       'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors',
       'VehicleIntroDate', 'AlarmImmobiliser', 'TrackingDevice',
       'CapitalOutstanding', 'NewVehicle', 'SumInsured', 'TermFrequency',
       'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory',
       'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass',
       'StatutoryRiskType', 'TotalPremium', 'TotalClaims',
       'Monthly_Premium_Change', 'Monthly_Claims_Change', 'HasClaim',
       'Margin'],
      dtype='object')